In [2]:
# Import Libraries #
import pandas as pd
import numpy as np
import os

In [3]:
###Python shortcut to folder path###
path = r'C:\Users\hazem\01-2023 Instacart Basket Analysis - Master Folder'

In [4]:
ords_prods_merged = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged.pkl'))

In [5]:
ords_prods_merged.groupby('product_name')

In [6]:
## Performing a Single Aggregation ##
ords_prods_merged.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


In [7]:
ords_prods_merged['max_order'] = ords_prods_merged.groupby(['user_id'])['order_number'].transform(np.max)

In [8]:
ords_prods_merged.head(15)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,price_range_loc,busiest_day,busy_days,busiest_period_of_day,max_order
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.00000,5,0,both,Mid-range product,Regularly busy,Neutral days,Most orders,32
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.00000,1,1,both,Mid-range product,Regularly busy,Neutral days,Most orders,32
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.00000,20,0,both,Mid-range product,Busiest day,Busiest days,Average orders,5
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,11.11484,10,0,both,Mid-range product,Regularly busy,Slowest days,Most orders,3
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.00000,11,1,both,Mid-range product,Least busy,Slowest days,Most orders,3
5,1,Chocolate Sandwich Cookies,61,19,5.8,1701441,777,16,1,7,26.00000,7,0,both,Mid-range product,Regularly busy,Busiest days,Most orders,26
6,1,Chocolate Sandwich Cookies,61,19,5.8,1871483,825,3,2,14,30.00000,2,0,both,Mid-range product,Regularly busy,Neutral days,Most orders,9
7,1,Chocolate Sandwich Cookies,61,19,5.8,1290456,910,12,3,10,30.00000,1,0,both,Mid-range product,Regularly busy,Slowest days,Most orders,12
8,1,Chocolate Sandwich Cookies,61,19,5.8,369558,1052,10,1,20,19.00000,1,0,both,Mid-range product,Regularly busy,Busiest days,Average orders,20
9,1,Chocolate Sandwich Cookies,61,19,5.8,589712,1052,15,1,12,15.00000,2,1,both,Mid-range product,Regularly busy,Busiest days,Most orders,20


# creating a loyalty flag for existing customers #

In [9]:
ords_prods_merged.loc[ords_prods_merged['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [10]:
ords_prods_merged.loc[(ords_prods_merged['max_order'] <= 40) & (ords_prods_merged['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [11]:
ords_prods_merged.loc[ords_prods_merged['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [12]:
ords_prods_merged['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [13]:
ords_prods_merged[['user_id', 'loyalty_flag', 'order_number']].head(20)

,user_id,loyalty_flag,order_number
0,138,Regular customer,28
1,138,Regular customer,30
2,709,New customer,2
3,764,New customer,1
4,764,New customer,3
5,777,Regular customer,16
6,825,New customer,3
7,910,Regular customer,12
8,1052,Regular customer,10
9,1052,Regular customer,15


# ## Performing Multiple Aggregations to check how spending behavior differ between customers based on loyalflag #

In [14]:
ords_prods_merged.groupby('loyalty_flag').agg({'prices': ['mean','sum']})

prices             
                       mean          sum
loyalty_flag                            
Loyal customer    10.386336  106814042.2
New customer      13.294670   83011787.2
Regular customer  12.495717  198391693.2

#loyal customer seems have less average than new customers but at the end of the day they 
spend more , yet regular customer is the highest paid in Instacart#

# Create a spending flag for each user based on the average price #

In [15]:
#  Creating new column in order to create spending_flag using the transform() function #
ords_prods_merged['average_price'] = ords_prods_merged.groupby(['user_id'])['prices'].transform(np.mean)

In [16]:
ords_prods_merged.head(20)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,...,add_to_cart_order,reordered,_merge,price_range_loc,busiest_day,busy_days,busiest_period_of_day,max_order,loyalty_flag,average_price
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,5,0,both,Mid-range product,Regularly busy,Neutral days,Most orders,32,Regular customer,6.935811
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,1,1,both,Mid-range product,Regularly busy,Neutral days,Most orders,32,Regular customer,6.935811
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,20,0,both,Mid-range product,Busiest day,Busiest days,Average orders,5,New customer,7.930208
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,10,0,both,Mid-range product,Regularly busy,Slowest days,Most orders,3,New customer,4.972414
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,11,1,both,Mid-range product,Least busy,Slowest days,Most orders,3,New customer,4.972414
5,1,Chocolate Sandwich Cookies,61,19,5.8,1701441,777,16,1,7,...,7,0,both,Mid-range product,Regularly busy,Busiest days,Most orders,26,Regular customer,6.935398
6,1,Chocolate Sandwich Cookies,61,19,5.8,1871483,825,3,2,14,...,2,0,both,Mid-range product,Regularly busy,Neutral days,Most orders,9,New customer,5.957576
7,1,Chocolate Sandwich Cookies,61,19,5.8,1290456,910,12,3,10,...,1,0,both,Mid-range product,Regularly busy,Slowest days,Most orders,12,Regular customer,6.680000
8,1,Chocolate Sandwich Cookies,61,19,5.8,369558,1052,10,1,20,...,1,0,both,Mid-range product,Regularly busy,Busiest days,Average orders,20,Regular customer,7.162500
9,1,Chocolate Sandwich Cookies,61,19,5.8,589712,1052,15,1,12,...,2,1,both,Mid-range product,Regularly busy,Busiest days,Most orders,20,Regular customer,7.162500


In [17]:
# Create spending_flag using the loc() function #
ords_prods_merged.loc[ords_prods_merged['average_price'] < 10, 'spending_flag'] = 'Low spender'

In [18]:
ords_prods_merged.loc[ords_prods_merged['average_price'] >= 10, 'spending_flag'] = 'High spender'

In [19]:
# print the frequency of “spending_flag” column using the value_counts() function #
ords_prods_merged['spending_flag'].value_counts(dropna = False)

Low spender     31770614
High spender      634245
Name: spending_flag, dtype: int64

# determine frequent versus non-frequent customers by Creating an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column #

In [20]:
#  Creating new column in order to create order_frequency_flag using the transform() function #
ords_prods_merged['median_of_prior_orders'] = ords_prods_merged.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [21]:
ords_prods_merged.head(10)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,...,_merge,price_range_loc,busiest_day,busy_days,busiest_period_of_day,max_order,loyalty_flag,average_price,spending_flag,median_of_prior_orders
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,both,Mid-range product,Regularly busy,Neutral days,Most orders,32,Regular customer,6.935811,Low spender,8.00000
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,both,Mid-range product,Regularly busy,Neutral days,Most orders,32,Regular customer,6.935811,Low spender,8.00000
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,both,Mid-range product,Busiest day,Busiest days,Average orders,5,New customer,7.930208,Low spender,8.00000
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,both,Mid-range product,Regularly busy,Slowest days,Most orders,3,New customer,4.972414,Low spender,11.11484
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,both,Mid-range product,Least busy,Slowest days,Most orders,3,New customer,4.972414,Low spender,11.11484
5,1,Chocolate Sandwich Cookies,61,19,5.8,1701441,777,16,1,7,...,both,Mid-range product,Regularly busy,Busiest days,Most orders,26,Regular customer,6.935398,Low spender,11.00000
6,1,Chocolate Sandwich Cookies,61,19,5.8,1871483,825,3,2,14,...,both,Mid-range product,Regularly busy,Neutral days,Most orders,9,New customer,5.957576,Low spender,19.00000
7,1,Chocolate Sandwich Cookies,61,19,5.8,1290456,910,12,3,10,...,both,Mid-range product,Regularly busy,Slowest days,Most orders,12,Regular customer,6.680000,Low spender,6.00000
8,1,Chocolate Sandwich Cookies,61,19,5.8,369558,1052,10,1,20,...,both,Mid-range product,Regularly busy,Busiest days,Average orders,20,Regular customer,7.162500,Low spender,10.00000
9,1,Chocolate Sandwich Cookies,61,19,5.8,589712,1052,15,1,12,...,both,Mid-range product,Regularly busy,Busiest days,Most orders,20,Regular customer,7.162500,Low spender,10.00000


In [22]:
# Create order_frequency_flag using the loc() function #
ords_prods_merged.loc[ords_prods_merged['median_of_prior_orders'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'

In [23]:
ords_prods_merged.loc[(ords_prods_merged['median_of_prior_orders'] > 10) & (ords_prods_merged['median_of_prior_orders'] <= 20), 'order_frequency_flag'] = 'Regular customer'

In [24]:
ords_prods_merged.loc[ords_prods_merged['median_of_prior_orders'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

In [25]:
# print the frequency of “order_frequency_flag” column using the value_counts() function #
ords_prods_merged['order_frequency_flag'].value_counts(dropna = False)

Frequent customer        20535136
Regular customer          9168905
Non-frequent customer     2700818
Name: order_frequency_flag, dtype: int64

In [26]:
ords_prods_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_merged.pkl'))